## Importing necessary packages and data

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load necessary data
future_data = pd.read_excel('../data/fut_bond_data_FVM5_2025-02-25.xlsx', sheet_name=0)
bond_data = pd.read_excel('../data/fut_bond_data_FVM5_2025-02-25.xlsx', sheet_name=1)

### Data Handling & Global Variables

In [3]:
# Futures Price (F)
F = future_data.loc[future_data['field'] == 'px_last', 'FVM5 Comdty'].iloc[0]
F = float(F)

# Extract current quote date and delivery date
t_current = pd.to_datetime(future_data.loc[future_data['field'] == 'last_update_dt', 'FVM5 Comdty'].iloc[0])
T_delivery = pd.to_datetime(future_data.loc[future_data['field'] == 'last_tradeable_dt', 'FVM5 Comdty'].iloc[0])

# Time to Delivery (tau) in ACT/360
days_to_delivery = (T_delivery - t_current).days
tau = days_to_delivery / 360.0

# Display the global variables
display(pd.DataFrame({
    'Variable': ['Futures Price (F)', 'Days to Delivery', 'Tau (ACT/360)'],
    'Value': [F, days_to_delivery, round(tau, 6)]
}))

,Variable,Value
0,Futures Price (F),107.406250
1,Days to Delivery,124.000000
2,Tau (ACT/360),0.344444


## 1. Cheapest-to-Deliver Analysis

### 1.1 Delivery Cost

The **delivery cost** measures the financial cost to the short position to buy the bond and deliver it against the futures contract. It is defined as:

$$
\text{Delivery Cost}^i = P_t^i - \phi^i \times F(t,T)
$$

Where:

- $P_t^i$ : Clean price of the $i$-th bond  
- $\phi^i$ : Conversion factor for the $i$-th bond  
- $F(t,T)$ : Futures price  

The bond with the lowest delivery cost is called the **Cheapest to Deliver (CTD)**.

In [4]:
# Calculate Delivery Cost (Formula: P - (Conversion Factor * Futures Price))
bond_data['delivery_cost'] = bond_data['px_last'] - (bond_data['conversion'] * F)

# Identify the bond with the minimum delivery cost
min_cost_idx = bond_data['delivery_cost'].idxmin()
ctd_candidate_cost = bond_data.loc[min_cost_idx]

# Display the bond with the lowest delivery cost
display(pd.DataFrame({
    'Ticker': [ctd_candidate_cost['ticker']],
    'Delivery Cost': [round(ctd_candidate_cost['delivery_cost'], 6)]
}))

,Ticker,Delivery Cost
0,91282CLK Govt,-0.185922


### 1.2 Identify the CTD

The **Cheapest-to-Deliver (CTD)** bond is formally identified as the one with the lowest Delivery Price Ratio (or, similarly, the lowest basis). The delivery price ratio indicates the cost per unit of the conversion factor:

$$
\text{Delivery Price Ratio}^i = \frac{P_t^i}{\phi^i}
$$

where:
- $P_t^i$ is the clean price of the $i$-th bond  
- $\phi^i$ is the conversion factor of the $i$-th bond

We calculate this ratio for all bonds to confirm the CTD identity. We then compare our result to the `fut_ctd` field provided in the `future_data`.

In [5]:
# Calculate Delivery Price Ratio
bond_data['delivery_price_ratio'] = bond_data['px_last'] / bond_data['conversion']

# Identify CTD based on minimum ratio
ctd_idx = bond_data['delivery_price_ratio'].idxmin()
ctd_bond = bond_data.loc[ctd_idx]

# Display CTD Ticker and Delivery Price Ratio as a DataFrame
display(
    pd.DataFrame({
        'CTD Ticker': [ctd_bond['ticker']],
        'Delivery Price Ratio': [round(ctd_bond['delivery_price_ratio'], 6)]
    })
)

# Compare with Bloomberg Field
bbg_ctd = future_data.loc[future_data['field'] == 'fut_ctd', 'FVM5 Comdty'].iloc[0]
display(pd.DataFrame({'Bloomberg Reported CTD': [bbg_ctd]}))

,CTD Ticker,Delivery Price Ratio
0,91282CLK Govt,107.202723


,Bloomberg Reported CTD
0,T 3.625 08/31/29


**Question : Does this match the CTD identified by Bloomberg in the fut_ctd field?**

Answer: Yes, this matches the CTD identified by Bloomberg in the fut_ctd field, as 91282CLK Govt matures on 08/31/29 and has a 3.625% coupon—the same as the bond listed as CTD by Bloomberg.

## 2. Implied Repo Rate